<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/15_VNP46A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract night time lights from VIIRS

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# GEE layer ID
layer_name = "RS_048"

# Image reducer (options: mean, median, min, max, stdDev, sum, product)
img_col_reducer = "mean"

# Date parameters
start_year = 2018
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## VNP46A1: VIIRS Daily Gridded Day Night Band 500m



In [ ]:
lights = ee.ImageCollection("NOAA/VIIRS/001/VNP46A1") \
  .filterBounds(aoi)

## Night-time lights processing

In [ ]:
reducer_list = ee.Reducer.mean() \
.combine(reducer2 = ee.Reducer.median(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.min(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.max(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.sum(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.product(), sharedInputs=True)

def annual_image(year_date):
  start = ee.Date.fromYMD(year_date, 1, 1)
  end = ee.Date.fromYMD(year_date, 12, 31)
  date_range = ee.DateRange(start, end)
  # name = start.format('YYYY_MM').cat('_to_').cat(end.format('YYYY_MM'))
  name = year_date
  return lights \
        .filterDate(date_range) \
        .select('DNB_At_Sensor_Radiance_500m') \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'name': name})

annual_image = year_list.map(annual_image)

## Check an element of list
year_mosaic = ee.Image(annual_image.get(1))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())
print(year_mosaic.bandNames().getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
for i in  range(ee.List.length(annual_image).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_image.get(i))
  output_img = output_img.select(band_select)
  # output_name = f"{layer_name}_{aoi_name}_{img_col_reducer}_{ee.String(output_img.get('name')).getInfo()}"
  output_name = f"{layer_name}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"


  task = ee.batch.Export.image.toDrive(image = output_img,
                                      region = aoi.geometry(),
                                      description = "EXPORT IMAGE TO DRIVE",
                                      folder = "GEE_exports",
                                      fileNamePrefix = output_name,
                                      scale = 500,
                                      maxPixels = 10e12,
                                      crs = "EPSG:4326"
                                      )
  task.start()
  print("STARTED TASK ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)